In [ ]:
import logging
import asyncio
import pandas as pd
import polars as pl

from datetime import datetime, timedelta

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.config.enumerations import Channels
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections import Credentials, InfluxCredentials
from tastytrade.analytics.visualizations.custom import plot_live_candlesticks
from tastytrade.analytics.visualizations.charts import DynamicChart, Study


from tastytrade.messaging.processors.influxdb import TelegrafHTTPEventProcessor

from tastytrade.analytics.indicators.momentum import hull

# Show all rows in pandas DataFrames
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"
DURATION = 15

EDT = 5

start_time = datetime(2025, 1, 1)

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

## Test individual components

In [ ]:
# Set API credentials
credentials = Credentials(env=ENV)


credentials = Credentials(env="Live")
dxlink = DXLinkManager()

await dxlink.open(credentials=credentials)

In [3]:
influx_user = InfluxCredentials()
influxdb = influxdb_client.InfluxDBClient(
    url=InfluxCredentials().url, token=influx_user.token, org=influx_user.org
)
query_api = influxdb.query_api()

In [4]:
for handler in dxlink.router.handler.values():
    handler.add_processor(TelegrafHTTPEventProcessor())

In [ ]:
dxlink.router.handler[Channels.Candle].processors

In [14]:
symbol = "BTC/USD:CXTALP"
# symbol = "NVDA"
# symbol = "QQQ"
# symbol = "SPY"
# symbol = "SPX"

In [ ]:
start_time = datetime(2025, 2, 20, 12, 00)

for symbol in ["BTC/USD:CXTALP", "NVDA", "QQQ", "SPY", "SPX"]:

    for interval in ["1d", "1h", "30m", "15m", "5m", "1m"]:
        await dxlink.subscribe_to_candles(
            symbol=symbol,
            interval=interval,
            from_time=start_time,
        )

In [ ]:
dxlink.router.handler[Channels.Candle].processors["feed"].frames[f"{symbol}{{=m}}"].sort(
    "time", descending=True
).head(5)

In [22]:
df = dxlink.router.handler[Channels.Candle].processors["feed"].frames["NVDA{=m}"].to_pandas()

In [ ]:
df[df["count"] == 0]

In [ ]:
symbols = ["NVDA"]
symbols = ["SPX"]
symbols = ["SPX", "NVDA", "SPY", "QQQ"]
symbols = ["BTC/USD:CXTALP"]
symbols = ["BTC/USD:CXTALP", "SPX", "NVDA", "SPY", "QQQ"]
await dxlink.subscribe(symbols)

In [ ]:
await dxlink.unsubscribe(["BTC/USD:CXTALP"])

In [ ]:
dxlink.router.handler[Channels.Trade].processors["feed"].df.tail(10)

In [ ]:
dxlink.router.handler[Channels.Quote].processors["feed"].df.tail(10)

In [ ]:
dxlink.router.handler[Channels.Greeks].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Profile].processors["feed"].df

In [ ]:
dxlink.router.handler[Channels.Summary].processors["feed"].df

In [22]:
candle_symbol_1m = "NVDA{=m}"
candle_symbol_5m = "NVDA{=5m}"
candle_symbol_1m = "BTC/USD:CXTALP{=m}"
candle_symbol_5m = "BTC/USD:CXTALP{=5m}"
candle_symbol_1m = "SPX{=m}"
candle_symbol_5m = "SPX{=5m}"

In [ ]:
# Example usage
study_params = {"hull": {"length": 20, "price_col": "close"}}  # HMA period  # Price column to use

# Create the live chart with HMA
task = plot_live_candlesticks(
    dxlink=dxlink, symbol=candle_symbol_1m, start_time=datetime(2025, 2, 18, 9, 25)
)

In [ ]:
# Later when you want to stop
task.cancel()

In [ ]:
task = plot_live_candlesticks(dxlink, candle_symbol_5m, start_time=datetime(2025, 2, 18, 9, 25))

In [ ]:
task.cancel()

In [ ]:
candle_symbol_1m

In [ ]:
hull(dxlink, candle_symbol_1m).head(3)

In [23]:
hma_study = Study(
    name="HMA-20",
    compute_fn=hull,  # The hull function
    params={"length": 20},  # Parameters for hull function
    plot_params={
        "colors": {"Up": "#01FFFF", "Down": "#FF66FE"},  # Green for uptrend  # Red for downtrend
        "width": 2,
    },
    value_column="HMA",
    color_column="HMA_color",
)

In [ ]:
chart1 = DynamicChart(
    dxlink,
    candle_symbol_1m,
    start_time=pd.Timestamp("2025-02-18 09:00", tz="America/New_York")
    .tz_convert("UTC")
    .to_datetime64(),
    end_time=pd.Timestamp("2025-02-18 16:00", tz="America/New_York")
    .tz_convert("UTC")
    .to_datetime64(),
)
chart1.add_study(hma_study)
chart1.start()

In [27]:
await chart1.stop()

In [ ]:
chart2 = DynamicChart(
    dxlink,
    candle_symbol_5m,
    start_time=pd.Timestamp("2025-02-19 09:00", tz="America/New_York")
    .tz_convert("UTC")
    .to_datetime64(),
)
chart2.add_study(hma_study)
chart2.start()

In [ ]:
chart2.stop()

In [ ]:
await dxlink.close()

In [24]:
# TODOS

# [x] Widen the plot
# Remove the scroller at the bottom
# move the legent and remove Price (that is obvious)

# Add MACD
# Add RSI
# Add Volume Profile (?? ... /ES, SPY, etc)
# Add velocity metric

# Do not plot study data points passed available data

# [x] HULL - Align the first tick against to the subsequent tick

# ERROR if no study data found

In [ ]:
from tastytrade.messaging.processors.default import CandleEventProcessor
from tastytrade.messaging.models.events import BasicCandleEvent

candle_processor = CandleEventProcessor()
candle_processor.process_event(
    BasicCandleEvent(
        eventSymbol="SPX", time=datetime(2025, 2, 14, 10, 0), open=100, high=100, low=100, close=100
    )
)
candle_processor.df

In [ ]:
tasks = asyncio.all_tasks()

In [ ]:
import asyncio


def list_all_coroutines():
    """List all currently running coroutines."""
    tasks = asyncio.all_tasks()
    coroutines = [task.get_coro() for task in tasks]
    return coroutines


# Example usage
coroutines = list_all_coroutines()
for coro in coroutines:
    print(coro)

In [ ]:
import asyncio


def cancel_coroutine(coroutine_name: str):
    """Cancel a coroutine by its name."""
    tasks = asyncio.all_tasks()
    for task in tasks:
        coro = task.get_coro()
        if coroutine_name in str(coro):
            task.cancel()
            print(f"Cancelled coroutine: {coroutine_name}")
            return True
    print(f"Coroutine not found: {coroutine_name}")
    return False


# Example usage
cancel_coroutine("DynamicChart.update_chart at 0x7d6d2058b940")

In [ ]:
await dxlink.close()

In [ ]:
import os
from typing import Optional


def query_flux_measurement(
    event_type: str = "CandleEvent",
    symbol: str = "SPX{=m}",
    start: datetime = datetime(2024, 2, 18, 9 + 5, 0),
    stop: Optional[datetime] = None,
) -> pd.DataFrame:
    """Convert FluxRecord to dict, dropping internal InfluxDB fields"""
    if stop:
        date_range = (
            f"{start.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {stop.strftime('%Y-%m-%dT%H:%M:%SZ')})"
        )
    else:
        date_range = f"{start.strftime('%Y-%m-%dT%H:%M:%SZ')})"

    pivot_query = f"""
        from(bucket: "{os.environ["INFLUX_DB_BUCKET"]}")
        |> range(start: {date_range}
        |> filter(fn: (r) => r["_measurement"] == "{event_type}")
        |> filter(fn: (r) => r["eventSymbol"] == "{symbol}")
        |> pivot(
            rowKey: ["_time"],
            columnKey: ["_field"],
            valueColumn: "_value"
            )
        """

    print(pivot_query)

    tables = query_api.query(pivot_query, org=os.environ["INFLUX_DB_ORG"])

    records = []
    for table in tables:
        for record in table.records:
            record_dict = {k: v for k, v in record.values.items()}

            record_dict["time"] = record.get_time()
            records.append(record_dict)

    drop_columns = ["result", "table", "_start", "_stop", "_time", "_measurement"]

    return pl.DataFrame(records).drop(drop_columns).sort("time")


df = query_flux_measurement()

In [ ]:
df

In [ ]:
set(
    [
        "eventSymbol",
        "time",
        "eventFlags",
        "index",
        "sequence",
        "count",
        "open",
        "high",
        "low",
        "close",
        "volume",
        "bidVolume",
        "askVolume",
        "openInterest",
        "vwap",
        "impVolatility",
        "tradeDate",
        "tradeDateUTC",
        "tradeTime",
        "tradeTimeUTC",
        "prevOpen",
        "prevHigh",
        "prevLow",
        "prevClose",
        "prevDate",
        "prevTime",
    ],
) - set(df.columns)

In [ ]:
df.columns

In [ ]:
dxlink.router.handler[Channels.Candle].processors["feed"].df.tail(10)

In [12]:
candles = dxlink.router.handler[Channels.Candle].processors["feed"].df

In [ ]:
candles.groupby("eventSymbol")["time"].count().sort_values(ascending=False)

In [ ]:
candles.shape

In [ ]:
rows = candles.groupby("eventSymbol")["time"].count().sort_values(ascending=False)
records = {}
for row in rows.index:
    symbol = row.partition("{")[0]
    if symbol in records:
        records[symbol] += int(rows[row])
    else:
        records[symbol] = int(rows[row])

records

In [65]:
df = dxlink.router.handler[Channels.Candle].processors["feed"].symbol["SPX{=5m}"].to_pandas()

In [ ]:
date_fltr = df["tradeDate"] == "2025-02-19"
time_fltr = df["tradeTime"] == df.loc[date_fltr]["tradeTime"].max()
df.loc[date_fltr & time_fltr][
    ["tradeDate", "tradeTime", "open", "high", "low", "close", "prevDate", "prevTime"]
]

In [ ]:
df.loc[date_fltr]["tradeTime"].max()

In [17]:
from tastytrade.providers.market import MarketDataProvider

provider = MarketDataProvider(dxlink, influxdb)

In [ ]:
provider.retrieve(
    symbol=f"{symbol}{{=5m}}",
    event_type="CandleEvent",
    start=datetime(2025, 2, 18),
)

await provider.subscribe(f"{symbol}{{=5m}}")

In [ ]:
provider[f"{symbol}{{=m}}"].sort("time", descending=True).head(2)

In [ ]:
provider[f"{symbol}{{=m}}"].sort("time", descending=True).tail(2)

In [ ]:
await dxlink.close()

In [ ]:
provider.frames.keys()

In [ ]:
provider["SPX{=5m}"]

In [27]:
from collections import Counter

In [ ]:
{key: val for key, val in Counter(provider["SPX{=m}"].to_pandas()["time"]).items() if val > 1}

In [ ]:
df

In [9]:
subs = await dxlink.get_active_subscriptions()

In [ ]:
subs.keys()